# LAMMPS Tutorials 03. Uniaxial tension simulation!

### Author: Mark Tschopp, mark.a.tschopp.civ at mail.mil

Please contact me if you have a problem with this tutorial, so I can modify in Github.  I have added FAQs, and will update my versions of LAMMPS in the future to keep the scripts current.

The latest version of this [Jupyter Notebook](http://ipython.org/notebook.html) tutorial is available at https://github.com/mrkllntschpp/lammps-tutorials.

The original tutorials are given here: https://icme.hpc.msstate.edu/mediawiki/index.php/LAMMPS_tutorials.  A number of these tutorials are out of date and have been ported over into the current iPython Jupyter Notebook tutorials on github.

***
## Abstract:
<a id="Sec1"></a>

This example script shows how to run an atomistic simulation of uniaxial tensile loading of an aluminum single crystal oriented in the $<100>$ direction. This example uses a parallel molecular dynamics code, LAMMPS [1]. These scripts were initially used to study dislocation nucleation in single crystal aluminum and copper [2-4].  This example uses the Mishin et al. [5] aluminum potential.  The deformation movies were created using OVITO [6] and AtomEye [7] visualization tools.

<table width="900" border="0" cellpadding="5">
<tr>

<td width="290" align="center" valign="center">
<figure>
  <img src="https://icme.hpc.msstate.edu/mediawiki/images/d/d7/Al_SC_100_stress-strain.jpg" width="300" title="Stress-strain curve for uniaxial tensile loading of single crystal aluminum">
</figure>
</td>

<td width="290" align="center" valign="center">
<figure>
  <img src="https://icme.hpc.msstate.edu/mediawiki/images/d/d6/Al_SC_100_disl.jpg" width="300" title="FCC Dislocation nucleation">
</figure>
</td>

<td width="290" align="center" valign="center">
<figure>
  <img src="https://icme.hpc.msstate.edu/mediawiki/images/8/84/Al_SC_100_movie1.gif" width="300" title="FCC Deformation Movie" />
</figure>
</td>
</tr>
</table>

***
## Complete Tutorials One and Two 

If you have not done so already, complete the first two tutorials available [here](https://github.com/mrkllntschpp/lammps-tutorials). 

***
## Description of Simulation 
This molecular dynamics simulation first generates a simulation cell with fcc atoms with $<100>$ orientations in the x, y, and z-directions. For this example, the simulation cell size is 10 lattice units in each direction, *i.e.*, 4000 total atoms. Larger simulation cell sizes should be used to converge the dislocation nucleation stress values and to not influence the dislocation nucleation mechanism. The potential used here is the Mishin *et al.* (1999) aluminum potential [5] used in Tutorials 1 and 2. The equilibration step allows the lattice to expand to a temperature of 300 K with a pressure of 0 bar at each simulation cell boundary. Then, the simulation cell is deformed in the x-direction at a strain rate of 10$^{10}$ s$^{-1}$, while the lateral boundaries are controlled using the NPT equations of motion to zero pressure. The stress and strain values are output to a separate file, which can be imported in a graphing application for plotting. The cfg dump files include the x, y, and z coordinates, the centrosymmetry values, the potential energies, and forces for each atom. This can be directly visualized using AtomEye [6]. 

## Step 1: Create the LAMMPS input script 
This input script was run using the January 2020 version of LAMMPS. Changes in some commands may require revision of the input script. To run this script, store it in `in.tensile.txt` and then use `lmp_exe < in.tensile.txt` in a UNIX environment where `lmp_exe` refers to the LAMMPS executable. 

In [1]:
%%writefile in.tensile.txt
# Input file for uniaxial tensile loading of single crystal aluminum
# Mark Tschopp
# To execute, use the syntax "lmp_exe < in.tensile.txt" from command prompt

# ------------------------ INITIALIZATION ----------------------------
units metal
dimension 3
boundary p p p
atom_style atomic
variable latparam equal 4.05

# ----------------------- ATOM DEFINITION ----------------------------
lattice fcc ${latparam}
region whole block 0 10 0 10 0 10
create_box 1 whole
lattice  fcc ${latparam} orient x 1 0 0 orient y 0 1 0 orient z 0 0 1
create_atoms 1 region whole

# ------------------------ FORCE FIELDS ------------------------------
pair_style eam/alloy
pair_coeff * * Al99.eam.alloy Al

# ------------------------- SETTINGS ---------------------------------
compute csym all centro/atom fcc
compute peratom all pe/atom 

######################################
# EQUILIBRATION
reset_timestep 0
timestep 0.001
velocity all create 300 12345 mom yes rot no
fix 1 all npt temp 300 300 1 iso 0 0 1 drag 1 

# Set thermo output
thermo 1000
thermo_style custom step lx ly lz press pxx pyy pzz pe temp

# Run for at least 10 picosecond (assuming 1 fs timestep)
run 20000
unfix 1

# Store final cell length for strain calculations
variable tmp equal "lx"
variable L0 equal ${tmp}
print "Initial Length, L0: ${L0}"

######################################
# DEFORMATION
reset_timestep 0

fix 1 all npt temp 300 300 1 y 0 0 1 z 0 0 1 drag 1
variable srate equal 1.0e10
variable srate1 equal "v_srate / 1.0e12"
fix 2 all deform 1 x erate ${srate1} units box remap x

# Output strain and stress info to file
# for units metal, pressure is in [bars] = 100 [kPa] = 1/10000 [GPa]
# p2, p3, p4 are in GPa
variable strain equal "(lx - v_L0)/v_L0"
variable p1 equal "v_strain"
variable p2 equal "-pxx/10000"
variable p3 equal "-pyy/10000"
variable p4 equal "-pzz/10000"
fix def1 all print 100 "${p1} ${p2} ${p3} ${p4}" file Al_SC_100.def1.txt screen no

# Use cfg for AtomEye
dump 1 all cfg 250 dump.tensile_*.cfg mass type xs ys zs c_csym c_peratom fx fy fz
dump_modify 1 element Al

# Display thermo
thermo 1000
thermo_style custom step v_strain temp v_p2 v_p3 v_p4 ke pe press

run 20000

######################################
# SIMULATION DONE
print "All done"

Overwriting in.tensile.txt


OK. Easy enough.  You should have the file in your working directory if you are running this from Jupyter Notebook.

***
## Step 2: Run the Simulation

Now run the simulation as we have done before.  On my computer, the 24Jan2020 LAMMPS executable is stored in the `C:\Program Files\LAMMPS 64-bit 24Jan2020\bin\` folder and is named `lmp_serial.exe`.  The `log.lammps` file should look like the output below. Notice that the 20 ps (20,000 fs) equilibration step brought the temperature of the simulation cell up to 300 K. The corresponding deformation went up to a strain of 0.2. The thermo command was set for every 1000 timesteps, so any plots should be made from the data included in the datafile.

<div class="alert alert-danger">
Warning! This next simulation can take a little time to execute on a laptop or desktop (approximately 15 minutes or so).  Here is where you start to appreciate the ability to use multiple processors to break up the spatial domain of your simulations.  And this is just a small toy simulation... 
</div>

In [2]:
!"C:/Program Files/LAMMPS 64-bit 20Apr2018/bin/lmp_serial" < in.tensile.txt

LAMMPS (20 Apr 2018)
OMP_NUM_THREADS environment is not set. Defaulting to 1 thread. (../comm.cpp:90)
  using 1 OpenMP thread(s) per MPI task
Lattice spacing in x,y,z = 4.05 4.05 4.05
Created orthogonal box = (0 0 0) to (40.5 40.5 40.5)
  1 by 1 by 1 MPI processor grid
Lattice spacing in x,y,z = 4.05 4.05 4.05
Created 4000 atoms
  Time spent = 0.00100088 secs
Neighbor list info ...
  update every 1 steps, delay 10 steps, check yes
  max neighbors/atom: 2000, page size: 100000
  master list distance cutoff = 8.28721
  ghost atom cutoff = 8.28721
  binsize = 4.1436, bins = 10 10 10
  2 neighbor lists, perpetual/occasional/extra = 1 1 0
  (1) pair eam/alloy, perpetual
      attributes: half, newton on
      pair build: half/bin/atomonly/newton
      stencil: half/bin/3d/newton
      bin: standard
  (2) compute centro/atom, occasional
      attributes: full, newton on
      pair build: full/bin/atomonly
      stencil: full/bin/3d
      bin: standard
Setting up Verlet run ...
  Unit style  

For this simulation, additional commands were added to produce files beyond just the `log.lammps` file.  Notice that there are `dump.tensile_XXXX.cfg` files (for visualization purposes) and a LAMMPS datafile `Al_SC_100.def1.txt` (short for Al/Aluminum, SC/single crystal, 100 for $<100>$ tensile direction.  These were the product of including the following lines to the input script:

`fix def1 all print 100 "${p1} ${p2} ${p3} ${p4}" file Al_SC_100.def1.txt screen no`

`dump 1 all cfg 250 dump.tensile_*.cfg mass type xs ys zs c_csym c_peratom fx fy fz`

For instance, here is what they look like:

In [3]:
# If this works, that would be great: !cat Al_SC_100.def1.txt
f = open('Al_SC_100.def1.txt', 'r')
file_contents = f.read()
print (file_contents)
f.close()

# Fix print output for fix def1
0.00099999999999895 0.0623001515122015 -0.00231508037939855 0.0176458093954208
0.00199999999999895 0.108174728813019 -0.0191399810633923 -0.031495011166045
0.00299999999999895 0.190580750973412 0.0184412175353795 0.0571545445852481
0.00399999999999895 0.220162702270795 -0.0175245588464412 -0.0100612035849307
0.00499999999999895 0.344278402166708 0.0308109249045193 0.0490591802645521
0.00599999999999894 0.357010367162386 -0.0165862957817257 -0.0106496375969664
0.00699999999999894 0.452837616967344 0.022441254954106 -0.00976318413997775
0.00799999999999894 0.493444912162597 -0.0218961145459516 0.00508025494057849
0.00899999999999894 0.5617490258727 0.0037745441470258 -0.0103329373913353
0.00999999999999894 0.631750028837068 0.0248880462097861 -0.0257986656112494
0.0109999999999989 0.705545033801963 -0.0290337466431727 -0.00432050316837468
0.0119999999999989 0.788679758673579 -0.00413711665030959 -0.00236955649877114
0.0129999999999989 0.838502716161917 -0.

In [4]:
# If this works, that would be great: !cat dump.tensile_0.cfg
f = open('dump.tensile_0.cfg', 'r')
file_contents = f.read()
print (file_contents)
f.close()

Number of particles = 4000
A = 1 Angstrom (basic length-scale)
H0(1,1) = 40.65 A
H0(1,2) = 0 A 
H0(1,3) = 0 A 
H0(2,1) = 0 A 
H0(2,2) = 40.65 A
H0(2,3) = 0 A 
H0(3,1) = 0 A 
H0(3,2) = 0 A 
H0(3,3) = 40.65 A
.NO_VELOCITY.
entry_count = 8
auxiliary[0] = c_csym
auxiliary[1] = c_peratom
auxiliary[2] = fx
auxiliary[3] = fy
auxiliary[4] = fz
26.982000 
Al 
0.0512667 0.0463079 0.00466241 1.4445 -3.31478 0.0955653 0.474951 -0.536391 
26.982000 
Al 
0.00282149 0.0494785 0.0488832 0.535203 -3.32473 -0.294961 -0.226 0.254706 
26.982000 
Al 
0.101154 0.0504284 0.0518532 0.270603 -3.28803 -0.446281 -0.179372 -0.292333 
26.982000 
Al 
0.0482332 0.100259 0.0518776 0.215115 -3.34699 -0.0115395 -0.0526568 -0.354057 
26.982000 
Al 
0.100035 0.0997008 0.0019859 0.277653 -3.32563 -0.393359 0.0944573 0.0397937 
26.982000 
Al 
0.0502796 0.0483144 0.103125 0.636603 -3.30714 -0.159076 -0.0176063 -0.294408 
26.982000 
Al 
0.104151 0.000549682 0.10187 0.538826 -3.29561 -0.362814 -0.412186 -0.261866 
26.982000 


***
## Step 3: Post-Processing the Simulation 

So, now what?  Why don't we open the `Al_SC_100.def1.txt` file for a stress-strain curve, and then visualize the data from the cfg files to see what is happening with the atoms.

### Stress-Strain Plot Using MATLAB and Python

The stress-strain curve in Figure 1 can be generated using the following MATLAB script:

```OCTAVE
%% Analyze def1.txt files
% Plot the various responses

d = dir('*.def1.txt');
for i = 1:length(d)
    % Get data
    fname = d(i).name;
    A = importdata(fname);
    strain = A.data(:,1);
    stress = A.data(:,2:4);

    % Generate plot
    plot(strain,stress(:,1),'-or','LineWidth',2,'MarkerEdgeColor','r',...
                'MarkerFaceColor','r','MarkerSize',5),hold on
    plot(strain,stress(:,2),'-ob','LineWidth',2,'MarkerEdgeColor','b',...
                'MarkerFaceColor','b','MarkerSize',5),hold on
    plot(strain,stress(:,3),'-og','LineWidth',2,'MarkerEdgeColor','g',...
                'MarkerFaceColor','g','MarkerSize',5),hold on
    axis square
    ylim([0 10])
    set(gca,'LineWidth',2,'FontSize',24,'FontWeight','normal','FontName','Times')
    set(get(gca,'XLabel'),'String','Strain','FontSize',32,'FontWeight','bold','FontName','Times')
    set(get(gca,'YLabel'),'String','Stress (GPa)','FontSize',32,'FontWeight','bold','FontName','Times')
    set(gcf,'Position',[1 1 round(1000) round(1000)])

    % Export figure to tif file
    exportfig(gcf,strrep(fname,'.def1.txt','.tif'),'Format','tiff','Color','rgb','Resolution',300)
    close(1)
end
```

<figure>
  <img src="https://icme.hpc.msstate.edu/mediawiki/images/d/d7/Al_SC_100_stress-strain.jpg" width="300" title="Stress-strain curve for uniaxial tensile loading of single crystal aluminum">
  <figcaption><strong>Figure 1.</strong> Stress-strain curve for uniaxial tensile loading of single crystal aluminum in the $<100>$ loading direction.
The exportfig command refers to a script on MATLAB Central File website. You will need to go to the website and download the script into the current directory for this to work.</figcaption>
</figure>

Or, you can make something in Python:

In [5]:
import matplotlib.pyplot as plt
import csv

results = []
with open('Al_SC_100.def1.txt',newline='') as file:
    reader = csv.reader(file, delimiter=' ')
    next(reader)  # Skip header row.
    for row in reader:
        row2 = [float(i) for i in row]
        results.append(row2)
        print(row2)


[0.00099999999999895, 0.0623001515122015, -0.00231508037939855, 0.0176458093954208]
[0.00199999999999895, 0.108174728813019, -0.0191399810633923, -0.031495011166045]
[0.00299999999999895, 0.190580750973412, 0.0184412175353795, 0.0571545445852481]
[0.00399999999999895, 0.220162702270795, -0.0175245588464412, -0.0100612035849307]
[0.00499999999999895, 0.344278402166708, 0.0308109249045193, 0.0490591802645521]
[0.00599999999999894, 0.357010367162386, -0.0165862957817257, -0.0106496375969664]
[0.00699999999999894, 0.452837616967344, 0.022441254954106, -0.00976318413997775]
[0.00799999999999894, 0.493444912162597, -0.0218961145459516, 0.00508025494057849]
[0.00899999999999894, 0.5617490258727, 0.0037745441470258, -0.0103329373913353]
[0.00999999999999894, 0.631750028837068, 0.0248880462097861, -0.0257986656112494]
[0.0109999999999989, 0.705545033801963, -0.0290337466431727, -0.00432050316837468]
[0.0119999999999989, 0.788679758673579, -0.00413711665030959, -0.00236955649877114]
[0.012999999

In [6]:
results2 = np.transpose(results)
plt.plot(results2[0],results2[1], '-or', label='Stress in X', lw=2, markersize = 5, mec = 'r', mfc = 'r')
plt.plot(results2[0],results2[2], '-ob', label='Stress in Y', lw=2, markersize = 5, mec = 'b', mfc = 'b')
plt.plot(results2[0],results2[3], '-og', label='Stress in Z', lw=2, markersize = 5, mec = 'g', mfc = 'g')
plt.xlabel('Strain',fontsize=16)
plt.ylabel('Stress (GPa)',fontsize=16)
plt.title('Stress versus Strain',fontsize=16)
plt.legend(fontsize=12)
plt.axis(aspect='equal')
plt.ylim(0,10)
plt.show()

NameError: name 'np' is not defined

***
## Step 4: Making the Deformation Movie 

OK, so there are a far greater number of different software visualization packages out there now than there were a decade ago.  I used to use AtomEye almost exclusively for visualization.  I then found ways to visualize things in MATLAB, where I could add a lot of different elements.  I would suggest downloading OVITO and trying it out, but there are a number of different packages.  The AtomEye and ImageJ thread of the tutorial is from a long time ago.  

### Visualization in Windows with OVITO

First download [OVITO](https://www.ovito.org/).
* Go to [OVITO](https://www.ovito.org/).
* Click on download, select operating system, and then create username to download.  Follow remaining instructions to get it on yoru computer.

OK.  Here goes:
* Open [OVITO](https://www.ovito.org/)
* Load file and select the `dump.tensile_0.cfg` file.  OVITO will find the rest of the `*.cfg` files.
* Drag the slider bar at the bottom to `64/80` because that is about where the peak of the stress-strain curve above appears.
* Now, go to the `Add modification...` pull down list and select `Color coding`.  Now go to the bottom `Particle property` pulldown list and select `c_csym`.  Now click on adjust range to get a start and end value that is reasonable for viewing the dislocation.
* Click on the play button below to view the evolution of the deformation process.
* To create a movie, click on the `Render Active Viewport` tab up above.  In the `Rendering range` box, select `Complete animation`.  Click on `Save to file` and click on the `Choose...` button (name, "deformation-movie.png").  I like transparent background.  Then click on `Render Active Viewport` at the top for it to render the animation.
* Use ImageJ or whatever you like to pull all the images together into an animated gif or the like to create a movie file.

As a side note:
* If you want, you can `Add Modification...` and select all atoms with low centrosymmetry and delete these atoms from the view.  Unfortunately, the size of the periodic cell is not large enough to be able to view the dislocation nucleation and growth process all that well.  See below for a video with a slightly larger cell size.

For instance, change this to increase the simulation cell size from 10 by 10 by 10 (1000 by 4 atoms = 4000 atoms):
`region whole block 0 10 0 10 0 10`

or double the dimension to increase the number of atoms to 32,000 atoms:
`region whole block 0 20 0 20 0 20`

or increase the dimension to 50 by 50 by 50 to increase the number of atoms to 500,000 atoms:
`region whole block 0 50 0 50 0 50`

<div class="alert alert-warning">
  <strong>Warning!</strong> Don't expect that this will finish on your laptop very quickly.  Here is where you are going to need multiple processors and/or a supercomputer.
</div>


### Visualization in UNIX with AtomEye

First download [AtomEye](http://mt.seas.upenn.edu/Archive/Graphics/A/). 
* Go to [AtomEye](http://mt.seas.upenn.edu/Archive/Graphics/A/) website at http://mt.seas.upenn.edu/Archive/Graphics/A/. 
* Click on Download, this will take you to the raw binary files. 
* Download the A.i686 version by right-clicking on the link and "Save Target As..." to one of your directories. 
* Rename the downloaded file as A by typing "cp A.i686 A". A will be the executable. 
* Using UNIX, run "chmod A 755" on the file to change this to an executable. 
* To test, you need to save a CFG file as well, such as cnt8x3.cfg. Try running using "./A cnt8x3.cfg". 

Next download [ImageJ](http://rsbweb.nih.gov/ij/). 
* Go to [ImageJ](http://rsbweb.nih.gov/ij/) website at http://rsbweb.nih.gov/ij/. 
* Download the appropriate version for Windows, LINUX, or MAC. 

This assumes that you already have AtomEye and ImageJ downloaded and installed. 
* Visualize the dumpfile in AtomEye by typing the following command, "/A dump.tensile_0.cfg" (UNIX). 
* Use the AtomEye options to select how you want to visualize deformation. In this example, the centrosymmetry parameter was used to show only atoms in a non-centrosymmetric environment (see Fig. 2). 
    * Use Alt+0 to activate centrosymmetric (csym) view. 
    * Adjust threshold, or set of atoms to view, by using Shift+T. This will allow creation of a set for the current parameter (in this case, csym). Please note that you need to adjust both lower and higher thresholds unless the atoms from following images that exceeds maximum value for the first one will be not shown. You can make it 5 or 10. 
    * Make atoms with values outside of the threshold invisible by using Ctrl+A. 
* Press 'y' within AtomEye to produce an animation script. 
* The folder "Jpg" now contains snapshots of all dumpfiles. 
* Open ImageJ 
* Drag the folder Jpg into ImageJ 
    * Select "Convert to RGB" to keep the color from the AtomEye images. 
    * Choose "yes" to load a stack. 
* Adjust the size as needed (Image/Adjust/Size) 
* Adjust frame rate as desired (Image/Stacks/Tools/Animation Options) 
* Save as Animated Gif file 
 

<table width="800" border="0" cellpadding="5">
<tr>

<td width="350" align="center" valign="center">
<figure>
  <img src="https://icme.hpc.msstate.edu/mediawiki/images/d/d6/Al_SC_100_disl.jpg" width="300" alt="Face-centered Cubic Lattice Structure" title="FCC Lattice Structure">
    <figcaption><strong>Figure 2.</strong> Image of nucleated dislocation near peak stress.</figcaption>
</figure>
</td>

<td width="350" align="center" valign="center">
<figure>
  <img src="https://icme.hpc.msstate.edu/mediawiki/images/8/84/Al_SC_100_movie1.gif" width="300" alt="Face-centered Cubic Lattice Structure" title="FCC Lattice Structure" />
  <figcaption><strong>Figure 3.</strong> Tensile Loading of an Aluminum Single Crystal. Movie showing deformation of single crystal aluminum loaded in the $<100>$ direction at a strain rate of 10$^{10}$ s$^{-1}$ and a temperature of 300 K. Only atoms in non-centrosymmetric environment are shown.
  </figcaption>
</figure>
</td>
</tr>
</table>

***
## FAQs 

<div class="alert alert-danger">
    <strong>Q1</strong>: I ran this tutorial of an atomistic simulation of uniaxial tensile loading of an aluminum single crystal oriented in the $<100>$ direction. However, I want to use the LJ potential instead of the EAM potential. When I run the input script with the Lennard Jones potential, I get much different results. The stress is nearly 3 times the stress from the EAM potential. I don't know where something is wrong. 
</div>

A1: Interatomic potentials play a commanding role in the properties of molecular dynamics simulations. The most simple potential is the Lennard Jones potential. The embedded atom method potential is a little more sophisticated and I would encourage you to look up the original papers by Mike Baskes (1983, 1984) to examine how this common potential formulation was created. While there are an increasing number of potential formulations out there now and an increasing number of potentials, the embedded atom method has been widely used over the years and still is. However, this brings up the point that different potentials may give vastly different properties because each potential may have been formulated for a particular problem. It is up to the user to decide whether the potential seems to give plausible results for their application or whether the initial fitting contains properties that are important for their application. 

***
## Links

* [Click here to open Tutorial 1](LAMMPS-Tutorials-01.ipynb)
* [Click here to open Tutorial 2](LAMMPS-Tutorials-02.ipynb)
* [Click here to open the next tutorial](LAMMPS-Tutorials-04.ipynb)

***
## References 

1. S. Plimpton, "Fast Parallel Algorithms for Short-Range Molecular Dynamics," J. Comp. Phys., 117, 1-19 (1995). 
1. Spearot, D.E., Tschopp, M.A., Jacob, K.I., McDowell, D.L., "Tensile strength of <100> and <110> tilt bicrystal copper interfaces," Acta Materialia 55 (2007) p. 705-714 (http://dx.doi.org/10.1016/j.actamat.2006.08.060). 
1. Tschopp, M.A., Spearot, D.E., McDowell, D.L., "Atomistic simulations of homogeneous dislocation nucleation in single crystal copper," Modelling and Simulation in Materials Science and Engineering 15 (2007) 693-709 (http://dx.doi.org/10.1088/0965-0393/15/7/001). 
1. Tschopp, M.A., McDowell, D.L., "Influence of single crystal orientation on homogeneous dislocation nucleation under uniaxial loading," Journal of Mechanics and Physics of Solids 56 (2008) 1806-1830. (http://dx.doi.org/10.1016/j.jmps.2007.11.012). 
1. Y. Mishin, D. Farkas, M.J. Mehl, and D.A. Papaconstantopoulos, "Interatomic potentials for monoatomic metals from experimental data and ab initio calculations," Phys. Rev. B 59, 3393 (1999). 
1. A. Stukowski, "Visualization and analysis of atomistic simulation data with OVITO – the Open Visualization Tool," Modelling Simul. Mater. Sci. Eng. 18 (2010) 015012.
1. J. Li, "AtomEye: an efficient atomistic configuration viewer," Modelling Simul. Mater. Sci. Eng. 11 (2003) 173. 